In [2]:
import pandas as pd
df = pd.read_csv('data.csv')
df = df[df.IMPORTANT != 'YES'][['ENGLISH', 'FRENCH', 'THEME']]
str(df.to_dict())

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
import pandas as pd
inputs={'status':'Import'}
inputs={'status':'Export','id':'2','importance':'YES'}

In [ ]:
def api(inputs):
    if inputs['status'] == 'Import':
        df = pd.read_csv('data.csv')
        df = df[df.IMPORTANT != 'YES'][['ENGLISH', 'FRENCH', 'THEME']]
        return str(df.to_dict())
    elif inputs['status'] == 'Export':
        df = pd.read_csv('data.csv')
        df.at[int(inputs['id']), 'IMPORTANT'] = inputs['importance']
        df.to_csv('data.csv',index=False,header=True)
        return 'The dataframe has been updated in the line '+str(inputs['id'])
    else:
        return 'Wrong inputs, status = Import or Export'

In [ ]:
api(inputs)

'The dataframe has been updated in the line 2'

In [ ]:
df2=pd.read_csv('data.csv')

In [ ]:
from io import StringIO
from google.cloud import storage
from google.oauth2 import service_account
import json

bucket_name = 'flaskapicsv.appspot.com'
data_path = 'data.csv'

def export_data(bucket_name,data_path,data):
    GOOGLE_APPLICATION_CREDENTIALS='flaskapicsv-e0a051a4392b.json'
    with open(GOOGLE_APPLICATION_CREDENTIALS) as source:
        info = json.load(source)
    storage_credentials = service_account.Credentials.from_service_account_info(info)
    client = storage.Client(credentials=storage_credentials)
    bucket = client.get_bucket(bucket_name)
    data_blob = bucket.blob(data_path)
    data_blob.upload_from_string(
        data.to_csv(header=True,index=False), 'text/csv')

def import_data(bucket_name,data_path):
    GOOGLE_APPLICATION_CREDENTIALS='flaskapicsv-e0a051a4392b.json'
    with open(GOOGLE_APPLICATION_CREDENTIALS) as source:
        info = json.load(source)
    storage_credentials = service_account.Credentials.from_service_account_info(info)
    client = storage.Client(credentials=storage_credentials)
    bucket = client.get_bucket(bucket_name)
    data_blob = bucket.blob(data_path)
    data_str = data_blob.download_as_text()
    data_dowloaded_df = pd.read_csv(StringIO(data_str))
    return data_dowloaded_df

In [4]:
from flask import Flask, request
import pandas as pd
from io import StringIO
from google.cloud import storage
from google.oauth2 import service_account
import json


bucket_name = 'flaskapicsv.appspot.com'
data_path = 'dataset/'+'FRENCH'+'/'+'FRENCH'+'_'+'ENGLISH'+'.csv'
# --------------------------------------
def import_data(bucket_name, data_path):
    GOOGLE_APPLICATION_CREDENTIALS = 'flaskapicsv-e0a051a4392b.json'
    with open(GOOGLE_APPLICATION_CREDENTIALS) as source:
        info = json.load(source)
    storage_credentials = service_account.Credentials.from_service_account_info(
        info)
    client = storage.Client(credentials=storage_credentials)
    bucket = client.get_bucket(bucket_name)
    data_blob = bucket.blob(data_path)
    data_str = data_blob.download_as_text()
    data_dowloaded_df = pd.read_csv(StringIO(data_str))
    return data_dowloaded_df

df=import_data(bucket_name, data_path)

In [5]:
df

,ID,FRENCH,ENGLISH,INTERVAL
0,1944,accalmie,lull,91
1,1557,francaire,frenchmen,94
2,11207,demain,tomorrow,23
3,11173,équitation,riding,23
4,1211,pentacle,pentagram,96
...,...,...,...,...
5318,13180,infraction,offense,9
5319,7787,mutinerie,mutiny,48
5320,7359,velours,velvet,51
5321,1902,joie,jain,91
